# This notebook runs the models for Table 2 (lag 0) and Table 4 (lag 10+) with and without using APOE as a covariate

In [ ]:
# !pip install statsmodels
# #!pip install pandas==1.5.3
# !pip install lifelines==0.26.4

In [ ]:
# Imports here.
import numpy as np
import pandas as pd
import os
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.multitest import fdrcorrection
from lifelines import CoxPHFitter

import warnings
warnings.filterwarnings("ignore")

# Download files

In [ ]:
!dx download /data/files_for_cox/AD_with_icd10_APOE_MAY_05_2025.csv
!dx download /data/files_for_cox/DEM_with_icd10_APOE_MAY_05_2025.csv
!dx download /data/files_for_cox/PD_with_icd10_APOE_MAY_05_2025.csv

In [ ]:
# Removed ipratropium & salmeterol because they are a spray -- should be 535 now
codes = ['moxifloxacin', 'fexofenadine', 'lormetazepam', 'nizatidine', 'lacidipine', 'rifaximin', 'levomepromazine', 'losartan', 'metformin', 'colchicine', 'valproicacid', 'acetylcysteine', 'phenoxymethylpenicillin', 'erdosteine', 'montelukast', 'sildenafil', 'amiodarone', 'clomethiazole', 'gemfibrozil', 'alogliptin', 'mirtazapine', 'aclidiniumbromide', 'clomipramine', 'nebivolol', 'minoxidil', 'clonidine', 'alfuzosin', 'rizatriptan', 'mecysteine', 'rimonabant', 'naratriptan', 'paracetamolandmetoclopramide', 'orlistat', 'griseofulvin', 'telmisartan', 'pravastatin', 'zaleplon', 'piroxicam', 'apixaban', 'amphotericin', 'linagliptin', 'amlodipine', 'diltiazem', 'nortriptyline', 'potassiumchlorideandpotassiumbicarbonate', 'propantheline', 'hydrotalcite', 'alprazolam', 'indomethacin', 'ciclesonide', 'nabumetone', 'perindopril', 'digoxin', 'oxerutins', 'amisulpride', 'bisacodyl', 'carbocisteine', 'gabapentin', 'misoprostol', 'ticagrelor', 'apomorphine', 'tetrabenazine', 'tetracycline', 'nalidixicacid', 'methylcellulose', 'flucloxacillin', 'epogam', 'mefenamic', 'dantrolene', 'cyproheptadine', 'phenylephrine', 'terbutaline', 'fentanyl', 'fluoxetine', 'moxisylyte', 'zonisamide', 'doxepin', 'sodiumvalproate', 'thioridazine', 'clomifene', 'fesoterodine', 'clonazepam', 'orciprenaline', 'fusidicacid', 'metronidazole', 'tramadol', 'ibandronicacid', 'gliclazide', 'naltrexone', 'chlortalidone', 'ciprofloxacin', 'orphenadrine', 'sulpiride', 'aminophylline', 'aspirin', 'glimepiride', 'budesonide', 'eprosartan', 'carvedilol', 'rupatadine', 'acemetacin', 'ibuprofen', 'aceclofenac', 'ketorolac', 'glyceryltrinitrate', 'olsalazine', 'zafirlukast', 'flurbiprofen', 'trimethoprim', 'cefixime', 'fosinopril', 'inositolnicotinate', 'entacapone', 'lercanidipine', 'acetazolamide', 'indapamide', 'metoprolol', 'febuxostat', 'captopril', 'disulfiram', 'propylthiouracil', 'tolbutamide', 'ezetimibe', 'risedronate', 'bisoprolol', 'pantoprazole', 'flavoxate', 'cinnarizine', 'nateglinide', 'hydrochlorothiazideandramipril', 'magnesium', 'eletriptan', 'trifluoperazine', 'chloramphenicol', 'alimemazine', 'moxonidine', 'oxcarbazepine', 'cetirizine', 'pseudoephedrine', 'albuterol', 'hydroxychloroquine', 'trimipramine', 'trimethoprimandsulfamethoxazole', 'nifedipine', 'sotalol', 'hyoscinebutylbromide', 'loratadine', 'erythromycin', 'naftidrofuryl', 'modafinil', 'esomeprazole', 'cisapride', 'oxytetracycline', 'tinidazole', 'sumatriptan', 'calcitriol', 'ranitidine', 'leflunomide', 'famciclovir', 'oxybutynin', 'perphenazine', 'ropinirole', 'ursodeoxycholicacid', 'citalopram', 'riluzole', 'darifenacin', 'buspirone', 'alendronatesodium', 'theophylline', 'paracetamolandcodeine', 'sulfasalazine', 'pre-exposureprophylaxis', 'lamotrigine', 'dexamethasone', 'stalevo', 'norfloxacin', 'ofloxacin', 'mianserin', 'nitrofurantoin', 'memantine', 'lorazepam', 'anticholinergic', 'midazolam', 'dronedarone', 'enalapril', 'fluticasone', 'lithium', 'mebendazole', 'diclofenac', 'quinapril', 'itraconazole', 'pyridostigmine', 'methylphenidate', 'trihexyphenidyl', 'co-amilofruse', 'mizolastine', 'irbesartan', 'glipizide', 'acamprosate', 'cyproterone', 'methotrexate', 'tolfenamicacid', 'prasugrel', 'mefloquine', 'methylprednisolone', 'isotretinoin', 'ibuprofenandcodeine', 'letrozole', 'nicardipine', 'magnesiumhydroxide', 'vancomycin', 'prednisone', 'potassiumchloride', 'saxagliptin', 'risperidone', 'dipipanone', 'tadalafil', 'astemizole', 'simvastatin', 'acarbose', 'quinine', 'levothyroxine', 'triamterene', 'nystatin', 'chlordiazepoxide', 'pramipexole', 'dihydrocodeine', 'allopurinol', 'terfenadine', 'mesalazine', 'hydrocortisonebuccaltablets', 'doxycycline', 'trandolapril', 'dicyclomine', 'asilone', 'nadolol', 'frovatriptan', 'ranolazine', 'azithromycin', 'cholestyramine', 'zopiclone', 'vigabatrin', 'tiotropium', 'bromocriptine', 'tiaprofenicacid', 'lidocaine', 'phenelzine', 'levofloxacin', 'diflunisal', 'hydralazine', 'acitretin', 'metoclopramideandaspirin', 'ciclosporin', 'methyldopa', 'tripotassiumdicitratobismuthate', 'galantamine', 'cyclobenzaprine', 'codeine', 'rifampicinandisoniazid', 'valaciclovir', 'phenytoin', 'co-amilozide', 'etoricoxib', 'sulindac', 'liothyronine', 'atenolol', 'cinacalcet', 'co-beneldopa', 'lansoprazole', 'indoramin', 'fenofibrate', 'zolmitriptan', 'hydrochlorothiazideandlisinopril', 'trospium', 'sucralfate', 'etamsylate', 'eplerenone', 'cromolynsodium', 'benzydamine', 'atorvastatin', 'cefalexin', 'minocycline', 'hydroxycarbamide', 'pioglitazone', 'rasagiline', 'carbamazepine', 'pivmecillinam', 'isosorbidedinitrate', 'formoterol', 'pentoxifylline', 'cabergoline', 'diazepam', 'furosemide', 'amiloride', 'tranexamicacid', 'ramipril', 'lofepramine', 'etidronicacid', 'promazine', 'aliskren', 'penicillamine', 'chlorpheniramine', 'prochlorperazine', 'bismuthsubsalicylate', 'cyclopenthiazide', 'finasteride', 'ironandfolicacid', 'penicillin', 'cyclizine', 'sertraline', 'ampicillin', 'dexketoprofen', 'nefazodone', 'solifenacin', 'felodipine', 'fluvoxamine', 'bupropion', 'tramacet', 'amoxicillin', 'tolterodine', 'isosorbidemononitrateandisosorbidedinitrate', 'betamethasone', 'cefradine', 'co-amoxiclav', 'clarithromycin', 'indometacin', 'almotriptan', 'escitalopram', 'chloroquine', 'ketotifen', 'oseltamivir', 'tamsulosin', 'olanzapine', 'levocetirizine', 'fluphenazine', 'chloralhydrate', 'tizanidine', 'cerivastatin', 'pholcodine', 'bezafibrate', 'methadone', 'bendroflumethiazide', 'methocarbamol', 'rivaroxaban', 'dosulepin', 'verapamil', 'mirabegron', 'salbutamol', 'amitriptyline', 'omeprazole', 'baclofen', 'fishoil', 'venlafaxine', 'meptazinol', 'cefpodoxime', 'fludrocortisone', 'levetiracetam', 'rosiglitazone', 'isoniazid', 'empagliflozin', 'carbimazole', 'lisinopril', 'doxazosin', 'co-magaldrox', 'temazepam', 'imipramine', 'gaviscon', 'mycophenolicacid', 'cefadroxil', 'methenamine', 'nicorandil', 'loperamide', 'mometasone', 'sterculia', 'cimetidine', 'hydroxyurea', 'celecoxib', 'meperidine', 'imidapril', 'terazosin', 'atropine', 'flecainide', 'thyroxine', 'hyoscinehydrobromide', 'dantronanddocusate', 'dabigatran', 'fluconazole', 'prucalopride', 'mebeverine', 'macrogol', 'lactulose', 'glibenclamide', 'sibutramine', 'meloxicam', 'valdecoxib', 'manevac', 'topiramate', 'ketoprofen', 'primidone', 'paroxetine', 'acrivastine', 'sodiumfusidate', 'strontium', 'zolpidem', 'oxprenolol', 'sulfinpyrazone', 'carisoprodol', 'pizotifen', 'clemastine', 'rivastigmine', 'bumetanide', 'agomelatine', 'famotidine', 'reboxetine', 'pregabalin', 'alprostadil', 'haloperidol', 'candesartan', 'kaolin', 'ivabradine', 'linaclotide', 'atenololandnifedipine', 'repaglinide', 'selegiline', 'olmesartan', 'oxazepam', 'paracetamol', 'dutasteride', 'prazosin', 'sodiumcromoglicate', 'amantadine', 'nefopam', 'mercaptopurine', 'insulin', 'docusate', 'buprenorphine', 'isosorbidemononitrate', 'chlorphenamine', 'diphenhydramine', 'valganciclovir', 'azathioprine', 'proguanil', 'propranolol', 'prednisolone', 'immunoglobin', 'metoclopramide', 'celiprolol', 'rofecoxib', 'trazodone', 'tamoxifen', 'raloxifene', 'moclobemide', 'metolazone', 'omega-3', 'dipyridamole', 'alverine', 'alendronicacid', 'quetiapine', 'duloxetine', 'simeticone', 'beclometasone', 'azapropazone', 'lymecycline', 'aripiprazole', 'domperidone', 'rabeprazole', 'hydrocortisone', 'phenobarbital', 'betahistine', 'hydroxyzine', 'loprazolam', 'prilocaine', 'triamcinolone', 'zoledronicacid', 'anastrozole', 'fybogel', 'canagliflozin', 'clopidogrel', 'balsalazide', 'vardenafil', 'lumiracoxib', 'labetalol', 'brompheniramine', 'danazol', 'clodronicacid', 'warfarin', 'methysergide', 'hydrochlorothiazide', 'sodiumpicosulfate', 'ondansetron', 'adalimumab', 'rifampicin', 'tenoxicam', 'colesevelam', 'ciprofibrate', 'chlorpromazine', 'pancrelipase', 'co-danthrusate', 'disopyramide', 'oxycodone', 'valsartan', 'nitrazepam', 'tacrolimus', 'lacosamide', 'flupentixol', 'cyclophosphamide', 'nitrousoxide', 'ethambutol', 'rosuvastatin', 'clobazam', 'donepezil', 'promethazine', 'spironolactone', 'co-careldopa', 'clindamycin', 'naproxen', 'senna', 'cefaclor', 'fluvastatin', 'propiverine', 'timolol', 'dutasterideandtamsulosin', 'dapagliflozin', 'migril', 'pilocarpine', 'aciclovir', 'morphine', 'sevelamer', 'etodolac', 'dapsone', 'sitagliptin', 'nedocromil', 'propafenone', 'oxitropium', 'cefuroxime']
print(len(codes))

In [ ]:
#Select NDDs
ndd_list = ['AD', 'DEM', 'PD']
print(len(codes))
print(ndd_list)

# Run Cox regressions without APOE

In [ ]:
timeline = 'ever taken'
drug = 'all'
model = f'icd10_{drug}'

# Change which lag you'd like to use here -- options are '0' (Table 2) or '10+' (Table 4)
lag_list = ['0']

results = []
cov_list = []

for lag in lag_list:

    for ndd in ndd_list:

        #Load df
        df = pd.read_csv(f'{ndd}_with_icd10_APOE_MAY_05_2025.csv', parse_dates = True)

        # Find codes to use so we don't have to use EVERYTHING
        codes_with_data = []

        for code in codes:

            m = df[['TOWNSEND', 'age_at_tenure', 'GENETIC_SEX', 'tenure', ndd, f'QC{lag}_' + code, 'QC0_A04', 'QC0_B02', 'QC0_B37', 'QC0_E10', 'QC0_E11', 'QC0_E27', 'QC0_E78', 'QC0_E87', 'QC0_F20', 'QC0_F31', 'QC0_F32', 'QC0_F33', 'QC0_F40', 'QC0_F41', 'QC0_F42', 'QC0_F43', 'QC0_F44', 'QC0_F45', 'QC0_F48', 'QC0_F50', 'QC0_F51', 'QC0_G40', 'QC0_G43', 'QC0_G47', 'QC0_H66', 'QC0_I10', 'QC0_I11', 'QC0_I12', 'QC0_I15', 'QC0_I20', 'QC0_I21', 'QC0_I25', 'QC0_I47', 'QC0_I48', 'QC0_I49', 'QC0_I50', 'QC0_I60', 'QC0_I61', 'QC0_I62', 'QC0_I63', 'QC0_I64', 'QC0_I65', 'QC0_I66', 'QC0_I67', 'QC0_I69', 'QC0_I82', 'QC0_K04', 'QC0_K05', 'QC0_K20', 'QC0_K21', 'QC0_K22', 'QC0_K25', 'QC0_K31', 'QC0_K51', 'QC0_K59', 'QC0_K70', 'QC0_K71', 'QC0_K72', 'QC0_K73', 'QC0_K74', 'QC0_K75', 'QC0_K76', 'QC0_L25', 'QC0_L40', 'QC0_L50', 'QC0_M06', 'QC0_M13', 'QC0_M15', 'QC0_M16', 'QC0_M17', 'QC0_M18', 'QC0_M19', 'QC0_M32', 'QC0_M45', 'QC0_M79', 'QC0_M80', 'QC0_M81', 'QC0_M88', 'QC0_N04', 'QC0_N10', 'QC0_N18', 'QC0_N19', 'QC0_N30', 'QC0_N31', 'QC0_N32', 'QC0_N39', 'QC0_N40', 'QC0_N94']]
            
            n=sum(m[f'QC{lag}_'+ code])
            df_pair = m[m[f'QC{lag}_'+ code]==1]
            n_pairs = sum(df_pair[ndd])
            if n == 0:
                pass
            elif n_pairs < 10:
                pass
            elif n == n_pairs:
                pass
            else:
                print(code)
                codes_with_data.append(code)

        print(ndd)
        print(len(codes_with_data))

        for code in codes_with_data:   

            m = df[['TOWNSEND', 'age_at_tenure', 'GENETIC_SEX', 'tenure', ndd, f'QC{lag}_' + code, 'QC0_A04', 'QC0_B02', 'QC0_B37', 'QC0_E10', 'QC0_E11', 'QC0_E27', 'QC0_E78', 'QC0_E87', 'QC0_F20', 'QC0_F31', 'QC0_F32', 'QC0_F33', 'QC0_F40', 'QC0_F41', 'QC0_F42', 'QC0_F43', 'QC0_F44', 'QC0_F45', 'QC0_F48', 'QC0_F50', 'QC0_F51', 'QC0_G40', 'QC0_G43', 'QC0_G47', 'QC0_H66', 'QC0_I10', 'QC0_I11', 'QC0_I12', 'QC0_I15', 'QC0_I20', 'QC0_I21', 'QC0_I25', 'QC0_I47', 'QC0_I48', 'QC0_I49', 'QC0_I50', 'QC0_I60', 'QC0_I61', 'QC0_I62', 'QC0_I63', 'QC0_I64', 'QC0_I65', 'QC0_I66', 'QC0_I67', 'QC0_I69', 'QC0_I82', 'QC0_K04', 'QC0_K05', 'QC0_K20', 'QC0_K21', 'QC0_K22', 'QC0_K25', 'QC0_K31', 'QC0_K51', 'QC0_K59', 'QC0_K70', 'QC0_K71', 'QC0_K72', 'QC0_K73', 'QC0_K74', 'QC0_K75', 'QC0_K76', 'QC0_L25', 'QC0_L40', 'QC0_L50', 'QC0_M06', 'QC0_M13', 'QC0_M15', 'QC0_M16', 'QC0_M17', 'QC0_M18', 'QC0_M19', 'QC0_M32', 'QC0_M45', 'QC0_M79', 'QC0_M80', 'QC0_M81', 'QC0_M88', 'QC0_N04', 'QC0_N10', 'QC0_N18', 'QC0_N19', 'QC0_N30', 'QC0_N31', 'QC0_N32', 'QC0_N39', 'QC0_N40', 'QC0_N94']]
            
            n=sum(m[f'QC{lag}_'+ code])
            df_pair = m[m[f'QC{lag}_'+ code]==1]
            n_pairs = sum(df_pair[ndd])

            cph = CoxPHFitter()
            cph.fit(m, duration_col = 'tenure', event_col = ndd, show_progress=False, step_size = 0.01)
            #cph.print_summary()
            #cph.plot()

            actual_p = cph._compute_p_values()
            results_df = cph.summary
            
            # Filter for significant covariates, e.g., p-value < 0.05
            significant_covariates = results_df[results_df['p'] < 0.05].index.tolist()
            #cov_list.append(code)
            cov_list.append(significant_covariates)
            
            results_df = results_df.reset_index()
            test = results_df.iloc[3]

            covariate = code
            HR = test['exp(coef)']
            ci_min = test['exp(coef) lower 95%']
            ci_max = test['exp(coef) upper 95%']
            p = actual_p[3]

            print(covariate, ndd, HR, ci_min, ci_max, p, n_pairs, n)
            results.append((covariate, ndd, model, timeline, lag, HR, ci_min, ci_max, p, n_pairs, n))
            
cox1 = pd.DataFrame(results, columns=('PRIOR','OUTCOME', 'MODEL','TIMELINE', 'LAG', 'HR', 'ci_min', "ci_max", 'P_VAL', "N_pairs", "N"))

In [ ]:
#Combine results
output = pd.concat([cox1])

#Adding FDR Correction

#Sort P-values
output = output.sort_values(by = "P_VAL")

#Drop Nan-values
output = output.dropna()

#FDR Correction
rejected, p_corr = fdrcorrection(output['P_VAL'], is_sorted=True)
output['P_CORR'] = p_corr
output['SIGNIFICANT'] = rejected

output

In [ ]:
# Note: Change file names to save lag10 files
output.to_csv(f'UKB_MAY_05_2025_results_UPDATED_with_ICD10_lag0.csv', header = True, index = False)

In [ ]:
! dx upload UKB_MAY_05_2025_results_UPDATED_with_ICD10_lag0.csv --path data/results/UKB_MAY_05_2025_results_UPDATED_with_ICD10_lag0.csv

In [ ]:
#cov_list

In [ ]:
from collections import Counter

# Example nested list
nested_list = cov_list

# Flatten the nested list
flat_list = [item for sublist in nested_list for item in sublist]

# Count the items
counts = Counter(flat_list)

# Convert to a DataFrame
counts_df = pd.DataFrame(counts.items(), columns=['item', 'count'])

counts_df

In [ ]:
counts_df.to_csv(f'UKB_MAY_05_2025_cox_covariates_lag0.csv', header = True, index = False)

In [ ]:
! dx upload UKB_MAY_05_2025_cox_covariates_lag0.csv --path data/results/UKB_MAY_05_2025_cox_covariates_lag0.csv

# Fix files to only include people for whom we have APOE data

In [ ]:
# Load previously created files -- repeat for all NDDs
ndd = 'AD'
#ndd = 'PD'
#ndd = 'DEM'
df = pd.read_csv(f'{ndd}_with_icd10_APOE_MAY_05_2025.csv', parse_dates = True)
print(len(df))
df.head()

In [ ]:
# Check APOE data
df.APOE_GENOTYPE.value_counts(dropna = False)

In [ ]:
#Remove samples with no APOE data
df = df[~df['APOE_GENOTYPE'].isna()]

#Remove samples with unknown
df = df[df['APOE_GENOTYPE'] != 'unknown']
df

In [ ]:
df.to_csv(f'{ndd}_removed_no_APOE_data_MAY_12_2025.csv', header = True, index=False)

In [ ]:
!dx upload {ndd}_removed_no_APOE_data_MAY_12_2025.csv --path /data/files_for_cox/{ndd}_removed_no_APOE_data_MAY_12_2025.csv

# Run Cox regressions -- WITH APOE status

In [ ]:
#Select NDDs
ndd_list = ['AD', 'DEM', 'PD']
print(len(codes))
print(ndd_list)

In [ ]:
timeline = 'ever taken'
drug = 'all'
model = f'icd10_{drug}'

# Change which lags you'd like to use here -- options are '0' (Table 2) or '10+' (Table 4)
lag_list = ['0']

results = []
cov_list = []

for lag in lag_list:

    for ndd in ndd_list:

        #Load df
        df = pd.read_csv(f'{ndd}_removed_no_APOE_data_MAY_12_2025.csv', parse_dates = True)

        # Find codes to use so we don't have to use EVERYTHING
        codes_with_data = []

        for code in codes:

            m = df[['TOWNSEND', 'age_at_tenure', 'GENETIC_SEX', 'tenure', ndd, f'QC{lag}_' + code, 'QC0_A04', 'QC0_B02', 'QC0_B37', 'QC0_E10', 'QC0_E11', 'QC0_E27', 'QC0_E78', 'QC0_E87', 'QC0_F20', 'QC0_F31', 'QC0_F32', 'QC0_F33', 'QC0_F40', 'QC0_F41', 'QC0_F42', 'QC0_F43', 'QC0_F44', 'QC0_F45', 'QC0_F48', 'QC0_F50', 'QC0_F51', 'QC0_G40', 'QC0_G43', 'QC0_G47', 'QC0_H66', 'QC0_I10', 'QC0_I11', 'QC0_I12', 'QC0_I15', 'QC0_I20', 'QC0_I21', 'QC0_I25', 'QC0_I47', 'QC0_I48', 'QC0_I49', 'QC0_I50', 'QC0_I60', 'QC0_I61', 'QC0_I62', 'QC0_I63', 'QC0_I64', 'QC0_I65', 'QC0_I66', 'QC0_I67', 'QC0_I69', 'QC0_I82', 'QC0_K04', 'QC0_K05', 'QC0_K20', 'QC0_K21', 'QC0_K22', 'QC0_K25', 'QC0_K31', 'QC0_K51', 'QC0_K59', 'QC0_K70', 'QC0_K71', 'QC0_K72', 'QC0_K73', 'QC0_K74', 'QC0_K75', 'QC0_K76', 'QC0_L25', 'QC0_L40', 'QC0_L50', 'QC0_M06', 'QC0_M13', 'QC0_M15', 'QC0_M16', 'QC0_M17', 'QC0_M18', 'QC0_M19', 'QC0_M32', 'QC0_M45', 'QC0_M79', 'QC0_M80', 'QC0_M81', 'QC0_M88', 'QC0_N04', 'QC0_N10', 'QC0_N18', 'QC0_N19', 'QC0_N30', 'QC0_N31', 'QC0_N32', 'QC0_N39', 'QC0_N40', 'QC0_N94', 'e3/e4', 'e4/e4']]
            
            n=sum(m[f'QC{lag}_'+ code])
            df_pair = m[m[f'QC{lag}_'+ code]==1]
            n_pairs = sum(df_pair[ndd])
            if n == 0:
                pass
            elif n_pairs < 10:
                pass
            elif n == n_pairs:
                pass
            else:
                print(code)
                codes_with_data.append(code)

        print(ndd)
        print(len(codes_with_data))

        for code in codes_with_data:   
           
            m = df[['TOWNSEND', 'age_at_tenure', 'GENETIC_SEX', 'tenure', ndd, f'QC{lag}_' + code, 'QC0_A04', 'QC0_B02', 'QC0_B37', 'QC0_E10', 'QC0_E11', 'QC0_E27', 'QC0_E78', 'QC0_E87', 'QC0_F20', 'QC0_F31', 'QC0_F32', 'QC0_F33', 'QC0_F40', 'QC0_F41', 'QC0_F42', 'QC0_F43', 'QC0_F44', 'QC0_F45', 'QC0_F48', 'QC0_F50', 'QC0_F51', 'QC0_G40', 'QC0_G43', 'QC0_G47', 'QC0_H66', 'QC0_I10', 'QC0_I11', 'QC0_I12', 'QC0_I15', 'QC0_I20', 'QC0_I21', 'QC0_I25', 'QC0_I47', 'QC0_I48', 'QC0_I49', 'QC0_I50', 'QC0_I60', 'QC0_I61', 'QC0_I62', 'QC0_I63', 'QC0_I64', 'QC0_I65', 'QC0_I66', 'QC0_I67', 'QC0_I69', 'QC0_I82', 'QC0_K04', 'QC0_K05', 'QC0_K20', 'QC0_K21', 'QC0_K22', 'QC0_K25', 'QC0_K31', 'QC0_K51', 'QC0_K59', 'QC0_K70', 'QC0_K71', 'QC0_K72', 'QC0_K73', 'QC0_K74', 'QC0_K75', 'QC0_K76', 'QC0_L25', 'QC0_L40', 'QC0_L50', 'QC0_M06', 'QC0_M13', 'QC0_M15', 'QC0_M16', 'QC0_M17', 'QC0_M18', 'QC0_M19', 'QC0_M32', 'QC0_M45', 'QC0_M79', 'QC0_M80', 'QC0_M81', 'QC0_M88', 'QC0_N04', 'QC0_N10', 'QC0_N18', 'QC0_N19', 'QC0_N30', 'QC0_N31', 'QC0_N32', 'QC0_N39', 'QC0_N40', 'QC0_N94', 'e3/e4', 'e4/e4']]
            
            n=sum(m[f'QC{lag}_'+ code])
            df_pair = m[m[f'QC{lag}_'+ code]==1]
            n_pairs = sum(df_pair[ndd])

            cph = CoxPHFitter()
            cph.fit(m, duration_col = 'tenure', event_col = ndd, show_progress=False, step_size = 0.01)
            #cph.print_summary()
            #cph.plot()

            actual_p = cph._compute_p_values()
            results_df = cph.summary
            
            # Filter for significant covariates, e.g., p-value < 0.05
            significant_covariates = results_df[results_df['p'] < 0.05].index.tolist()
            #cov_list.append(code)
            cov_list.append(significant_covariates)
            
            results_df = results_df.reset_index()
            test = results_df.iloc[3]

            covariate = code
            HR = test['exp(coef)']
            ci_min = test['exp(coef) lower 95%']
            ci_max = test['exp(coef) upper 95%']
            p = actual_p[3]

            print(covariate, ndd, HR, ci_min, ci_max, p, n_pairs, n)
            results.append((covariate, ndd, model, timeline, lag, HR, ci_min, ci_max, p, n_pairs, n))
            
cox1 = pd.DataFrame(results, columns=('PRIOR','OUTCOME', 'MODEL','TIMELINE', 'LAG', 'HR', 'ci_min', "ci_max", 'P_VAL', "N_pairs", "N"))

In [ ]:
#Combine results
output = pd.concat([cox1])

#Adding FDR Correction

#Sort P-values
output = output.sort_values(by = "P_VAL")

#Drop Nan-values
output = output.dropna()

#FDR Correction
rejected, p_corr = fdrcorrection(output['P_VAL'], is_sorted=True)
output['P_CORR'] = p_corr
output['SIGNIFICANT'] = rejected

output

In [ ]:
output.to_csv(f'UKB_MAY_12_2025_results_with_ICD10_and_APOE_lag0.csv', header = True, index = False)

In [ ]:
! dx upload UKB_MAY_12_2025_results_with_ICD10_and_APOE_lag0.csv --path data/results/UKB_MAY_12_2025_results_with_ICD10_and_APOE_lag0.csv

In [ ]:
from collections import Counter

# Example nested list
nested_list = cov_list

# Flatten the nested list
flat_list = [item for sublist in nested_list for item in sublist]

# Count the items
counts = Counter(flat_list)

# Convert to a DataFrame
counts_df = pd.DataFrame(counts.items(), columns=['item', 'count'])

counts_df

In [ ]:
counts_df.to_csv(f'UKB_MAY_12_2025_cox_covariates_with_APOE_lag0.csv', header = True, index = False)

In [ ]:
! dx upload UKB_MAY_12_2025_cox_covariates_with_APOE_lag0.csv --path data/results/UKB_MAY_12_2025_cox_covariates_with_APOE_lag0.csv